# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [13]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [14]:
weather_data = pd.read_csv ('weather_data.csv')
del weather_data["Unnamed: 0"]
weather_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Hasaki,35.7333,140.8333,41.40,93,100,17.40,JP,1644287316
1,Barrow,71.2906,-156.7887,-27.38,68,0,6.91,US,1644287016
2,Albany,42.6001,-73.9662,35.82,92,100,0.49,US,1644287123
3,Dunedin,-45.8742,170.5036,62.56,67,1,6.04,NZ,1644287317
4,Ushuaia,-54.8000,-68.3000,58.66,82,100,4.61,AR,1644286896
...,...,...,...,...,...,...,...,...,...
549,Jiazi,22.8779,116.0670,61.90,71,58,12.59,CN,1644287565
550,Acapulco de Juárez,16.8634,-99.8901,78.62,78,40,8.05,MX,1644287473
551,Maragogi,-9.0122,-35.2225,77.50,77,18,5.91,BR,1644287566
552,Al Bardīyah,31.7561,25.0865,55.38,51,82,18.28,LY,1644287567


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [15]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [16]:
# Store latitude and longitude in locations
location = weather_data[["Lat", "Lng"]]
humidity = weather_data['Humidity']

In [17]:
# Plot Heatmap
fig = gmaps.figure(center=(20.0, 10), zoom_level=2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(location, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2.5)

# Plot as a Terrain map
fig = gmaps.figure(map_type="TERRAIN")

# #trying out different gradients
# heat_layer.gradient = [
#     'white',
#     'yellow',
#     'red'
# ]

#Customize the size of the figure
figure_layout = {
    'width': '1000px',
    'height': '1000px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

fig = gmaps.figure(layout=figure_layout)

# Add layer
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(border='1px solid black', height='1000px', margin='0 auto 0 auto', padding='1px', w…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [18]:
weather_data_new = weather_data.loc[((weather_data['Max Temp'] <= 80) & (weather_data['Max Temp'] >= 70)) & ((weather_data['Wind Speed'] < 10) & (weather_data['Cloudiness'] == 0))]
hotel_df = weather_data_new
hotel_df.dropna()
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
175,Imbituba,-28.2400,-48.6703,73.29,64,0,5.01,BR,1644287391
187,Luquillo,18.3725,-65.7166,75.81,83,0,6.91,PR,1644287397
203,Ontario,34.0633,-117.6509,70.02,22,0,6.91,US,1644287201
266,Upington,-28.4478,21.2561,71.98,78,0,2.30,ZA,1644287434
267,Doctor Juan León Mallorquín,-25.4000,-55.2667,73.13,47,0,3.22,PY,1644287435
268,Alta Floresta,-9.8756,-56.0861,71.42,100,0,0.00,BR,1644287435
365,Mayor Pablo Lagerenza,-19.9309,-60.7718,74.71,46,0,6.08,PY,1644287480
442,Porto Velho,-8.7619,-63.9039,77.05,94,0,3.44,BR,1644287519
473,Saint-Pierre,-21.3393,55.4781,74.88,83,0,5.75,RE,1644287535
511,Te Anau,-45.4167,167.7167,72.18,39,0,2.53,NZ,1644287550


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [19]:
hotel_names = []
lat = hotel_df['Lat']
lng = hotel_df['Lng']
lat_lngs = zip(lat,lng)

for lat_lng in lat_lngs:

    target_coordinates = f"{lat_lng[0]},{lat_lng[1]}"
    target_search = "Hotel"
    target_radius = 5000
    target_type = "lodging"


    params = {
        "location": target_coordinates,
        "radius": target_radius,
        "types": target_type,
        "key": g_key
    }

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    response = requests.get(base_url, params=params)
       
    places_data = response.json()

    try:
        hotel_names.append(places_data["results"][1]["name"])
    except:
        print(f"No hotel found for {lat_lng}.")
        hotel_names.append("")

No hotel found for (-19.9309, -60.7718).


In [20]:
hotel_names

['Apartamentos Universo',
 'Wyndham Grand Rio Mar Puerto Rico Golf & Beach Resort',
 'Acacia Apartments',
 'Protea Hotel by Marriott Upington',
 'Hotel Hospeje Santa Librada',
 'Hotel Estoril',
 '',
 'Oscar Hotel Executive',
 'Villa Belza',
 'Radfords on the Lake',
 'Prasant Hotels']

In [21]:
hotel_df["Hotel Name"] = hotel_names
hotel_df


C:\Users\marqu\AppData\Local\Temp/ipykernel_19016/3497353961.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = hotel_names


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
175,Imbituba,-28.2400,-48.6703,73.29,64,0,5.01,BR,1644287391,Apartamentos Universo
187,Luquillo,18.3725,-65.7166,75.81,83,0,6.91,PR,1644287397,Wyndham Grand Rio Mar Puerto Rico Golf & Beach...
203,Ontario,34.0633,-117.6509,70.02,22,0,6.91,US,1644287201,Acacia Apartments
266,Upington,-28.4478,21.2561,71.98,78,0,2.30,ZA,1644287434,Protea Hotel by Marriott Upington
267,Doctor Juan León Mallorquín,-25.4000,-55.2667,73.13,47,0,3.22,PY,1644287435,Hotel Hospeje Santa Librada
268,Alta Floresta,-9.8756,-56.0861,71.42,100,0,0.00,BR,1644287435,Hotel Estoril
365,Mayor Pablo Lagerenza,-19.9309,-60.7718,74.71,46,0,6.08,PY,1644287480,
442,Porto Velho,-8.7619,-63.9039,77.05,94,0,3.44,BR,1644287519,Oscar Hotel Executive
473,Saint-Pierre,-21.3393,55.4781,74.88,83,0,5.75,RE,1644287535,Villa Belza
511,Te Anau,-45.4167,167.7167,72.18,39,0,2.53,NZ,1644287550,Radfords on the Lake


In [22]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [23]:
# Add marker layer on top of heat map
fig = gmaps.figure(center=(20.0, 10), zoom_level=2)

#create markers
markers = gmaps.marker_layer(locations)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

#trying out different gradients
# heat_layer.gradient = [
#     'white',
#     'purple',
#     'blue'
# ]

#Customize the size of the figure
figure_layout = {
    'width': '1000px',
    'height': '800px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

fig = gmaps.figure(layout=figure_layout)

#add heat layer
fig.add_layer(heat_layer)

#add marker layer
fig.add_layer(markers)

# Add info marker layer
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='800px', margin='0 auto 0 auto', padding='1px', wi…